----
# <center>Dataset creation

In [1]:
import os
from pathlib import Path
import matlab.engine
import numpy as np
import shutil
import tempfile
import json

base_dir = Path("./")

 # --- Start MATLAB Engine ---
eng = matlab.engine.start_matlab()

 # --- Add SCOPE path to MATLAB's search path ---
scope_path = './SCOPE'  # e.g., '/home/user/Documents/SCOPE'
eng.addpath(scope_path, nargout=0)

# MODTRAN paths
modtran_dir = base_dir / "MODTRAN5"
modtran_exe = modtran_dir / "bin" / "Mod5_mac.exe" 
modtran_tp5_template = modtran_dir / "HyPlant-FLUO_Modtran5_base_v1.tp5"

# SCOPE paths
scope_dir = base_dir / "SCOPE"
scope_main = scope_dir / "SCOPE.m"
scope_wrapper = scope_dir / "run_scope_wrapper.m"

# Output directory
output_dir = base_dir / "synthetic_dataset"
output_dir.mkdir(exist_ok=True)

print(f"Checking path: {scope_dir.resolve()}")
print(f"Directory exists: {scope_dir.exists()}")
print(f"Directory contents: {list(scope_dir.glob('*')) if scope_dir.exists() else 'N/A'}")


Checking path: /home/omirako/Documents/MSc_Sensors_Imaging/Final_Project/SCOPE
Directory exists: True
Directory contents: [PosixPath('SCOPE/soltir_tp7.m'), PosixPath('SCOPE/bug_reports.txt'), PosixPath('SCOPE/README.md'), PosixPath('SCOPE/output'), PosixPath('SCOPE/SCOPE.m'), PosixPath('SCOPE/set_parameter_filenames.csv'), PosixPath('SCOPE/input'), PosixPath('SCOPE/run_scope_wrapper_json.m'), PosixPath('SCOPE/.readthedocs.yaml'), PosixPath('SCOPE/src'), PosixPath('SCOPE/GNU_General_Public_Licence.txt'), PosixPath('SCOPE/SCOPE.exe'), PosixPath('SCOPE/docs')]


In [2]:
from scopeWrapper import SCOPEWrapperMultiRun

if __name__ == "__main__":
    print("\n=== Path Diagnostics ===")
    print(f"Current working directory: {Path.cwd()}")
    print(f"Checking for 'SCOPE' in: {Path.cwd()}")

    # the spectral and scalar files that will be saved in the results.json
    
    spectral_files = {
        #"Eout_spectrum": "Eout_spectrum.csv",
        "Esun": "Esun.csv",
        #"Lo_spectrum": "Lo_spectrum.csv", 
        "fluorescence": "fluorescence.csv",
        #"sigmaF": "sigmaF.csv",
        "reflectance": "reflectance.csv",
    }
    
    scalar_files = {
        #"aPAR": "aPAR.csv",
        #"Eout": "Eout.csv",
        #"Lo": "Lo.csv",
        #"Esun": "Esun.csv",
        #"Esky": "Esky.csv",
        #"fluxes": "fluxes.csv",
        #"rad": "radiation.csv",
        #"fluorescence_scalars": "fluorescence_scalars.csv",
    }

    try:
        with SCOPEWrapperMultiRun(scalar_files=scalar_files, spectral_files=spectral_files) as scope:
            print("\n=== Running Multi-Simulation Test ===")
            # Example parameters (can be adjusted)

            # Run a simulation with multiple LAI and Cab values
            multi_params = {
                # PROSPECT leaf optical properties
                "Cab": [30.0, 40.0, 50.0],      # Chlorophyll content (μg/cm²)
                "Cca": [10.0],                  # Carotenoid content (μg/cm²)
                "Cdm": [0.012],                 # Dry matter content (g/cm²)
                "Cw": [0.009],                  # Equivalent water thickness (cm)
                "Cs": [0.0],                    # Senescent material fraction (0-1)
                "Cant": [1.0],                  # Anthocyanin content (μg/cm²)
                "N": [1.5],                     # Leaf structure parameter (-)

                # Leaf biochemical parameters
                "Vcmax25": [60.0],              # Maximum carboxylation rate at 25°C (μmol/m²/s)
                "BallBerrySlope": [8.0],        # Ball-Berry stomatal conductance slope
                "BallBerry0": [0.01],           # Ball-Berry intercept (mol/m²/s)

                # Canopy structure
                "LAI": [2.0, 3.0, 5.0],        # Leaf Area Index (m²/m²)
                "hc": [2.0],                    # Canopy height (m)
                "LIDFa": [-0.35],               # Leaf angle distribution parameter a (-)
                "LIDFb": [-0.15],               # Leaf angle distribution parameter b (-) this one has to vary

                # Soil parameters
                "rss": [500.0],                 # Soil respiration rate (μmol/m²/s)
                "SMC": [25.0],                  # Soil moisture content (%)
                
                # Meteorology
                "Rin": [800.0],                 # Incoming shortwave radiation (W/m²)
                "Ta": [20.0],                   # Air temperature (°C)
                "Ca": [410.0],                  # Atmospheric CO₂ concentration (ppm)

                # Fluorescence
                "fqe": [0.01, 0.03, 0.06, 0.1, 0.2], # Fluorescence quantum efficiency (-)

                # Angles
                "tts": [35.0, 0.00, 60.0],                  # Solar zenith angle (degrees)
                "tto": [0.0],                   # Observer zenith angle (degrees) +- 20 degrees
                "psi": [0.0]                    # Relative azimuth angle (degrees)
            }

            my_setoptions = {
                "simulation": 2,  # Enable Lookup-Table mode
                "calc_fluor": 1,  # Calculate fluorescence
                "soil_heat_method": 2,  # Use soil net radiation fraction
                "saveCSV": 1,  # Output CSV files
                "calc_planck": 0,
                "calc_directional": 0,
                "calc_planck": 0,
                "calc_vert_profiles": 0,
                "calc_rss_rbs": 0,
                "soil_heat_method": 2,
                "save_spectral": 1,
                "calc_xanthophyllabs": 0,
                "applTcorr": 0,
                "MoninObukhov" : 0,
                "lite": 0,
                
            }

            results_path = scope.run(multi_params, my_setoptions)

            print("Success!")
            with open(results_path, "r") as f:
                results = json.load(f)

            print("\n=== Results ===")

            print(
                f"Number of simulations in output: {results.get('num_simulations')}"
            ) 

    except Exception as e:
        print(f"Error during SCOPE execution: {e}")


=== Path Diagnostics ===
Current working directory: /home/omirako/Documents/MSc_Sensors_Imaging/Final_Project
Checking for 'SCOPE' in: /home/omirako/Documents/MSc_Sensors_Imaging/Final_Project

=== Running Multi-Simulation Test ===
=== Parameters before CSV generation ===
{
  "Cab": [
    30.0,
    40.0,
    50.0
  ],
  "Cca": [
    10.0
  ],
  "Cdm": [
    0.012
  ],
  "Cw": [
    0.009
  ],
  "Cs": [
    0.0
  ],
  "Cant": [
    1.0
  ],
  "N": [
    1.5
  ],
  "Vcmax25": [
    60.0
  ],
  "BallBerrySlope": [
    8.0
  ],
  "BallBerry0": [
    0.01
  ],
  "LAI": [
    2.0,
    3.0,
    5.0
  ],
  "hc": [
    2.0
  ],
  "LIDFa": [
    -0.35
  ],
  "LIDFb": [
    -0.15
  ],
  "rss": [
    500.0
  ],
  "SMC": [
    25.0
  ],
  "Rin": [
    800.0
  ],
  "Ta": [
    20.0
  ],
  "Ca": [
    410.0
  ],
  "fqe": [
    0.01,
    0.03,
    0.06,
    0.1,
    0.2
  ],
  "tts": [
    35.0,
    0.0,
    60.0
  ],
  "tto": [
    0.0
  ],
  "psi": [
    0.0
  ]
}

=== Generated CSV Content ===
PRO

In [5]:
print(results.keys())
print(results["spectral_outputs"].keys())
print(results["input_parameters"].keys())
# print recursive keys
def print_keys(d, prefix=""):
    for k, v in d.items():
        print(f"{prefix}{k}")
        if isinstance(v, dict):
            print_keys(v, prefix=prefix + "-")
print_keys(results)

dict_keys(['scalar_outputs', 'run_parameters', 'num_simulations', 'input_parameters', 'setoptions', 'model_parameters', 'spectral_outputs', 'wlF', 'wlS'])
dict_keys(['fluorescence', 'reflectance'])
dict_keys(['PROSPECT', 'Leaf_Biochemical', 'Leaf_Biochemical_magnani', 'Fluorescence', 'Soil', 'Canopy', 'Meteo', 'Aerodynamic', 'timeseries', 'Angles'])
scalar_outputs
run_parameters
num_simulations
input_parameters
-PROSPECT
--Cab
--Cca
--Cdm
--Cw
--Cs
--Cant
--Cp
--Cbc
--N
--rho_thermal
--tau_thermal
-Leaf_Biochemical
--Vcmax25
--BallBerrySlope
--BallBerry0
--Type
--kV
--Rdparam
--Kn0
--Knalpha
--Knbeta
-Leaf_Biochemical_magnani
--Tyear
--beta
--kNPQs
--qLs
--stressfactor
-Fluorescence
--fqe
-Soil
--spectrum
--rss
--rs_thermal
--cs
--rhos
--lambdas
--SMC
--BSMBrightness
--BSMlat
--BSMlon
-Canopy
--LAI
--hc
--LIDFa
--LIDFb
--leafwidth
--Cv
--crowndiameter
-Meteo
--z
--Rin
--Ta
--Rli
--p
--ea
--u
--Ca
--Oa
-Aerodynamic
--zo
--d
--Cd
--rb
--CR
--CD1
--Psicor
--CSSOIL
--rbs
--rwc
-timeseries


In [8]:
import os
import logging
import shutil
import tempfile
import json
from pathlib import Path

import matlab.engine
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

results_path = "synthetic_dataset/results.json"
with open(results_path, "r") as f:
    scope_results = json.load(f)
scope_results['spectral_outputs']['Esun']

[[531.9595,
  499.8378,
  542.3789,
  508.6288,
  555.4373,
  496.9957,
  491.3637,
  510.2519,
  524.8139,
  580.9513,
  476.9882,
  556.3404,
  588.2313,
  563.2511,
  581.805,
  582.7686,
  573.0822,
  575.9477,
  565.791,
  578.9304,
  547.1223,
  624.1176,
  610.9692,
  532.4139,
  579.6995,
  596.5324,
  566.4143,
  559.8867,
  580.8906,
  551.7681,
  448.4648,
  421.0311,
  655.3339,
  594.8,
  541.3031,
  623.2561,
  660.833,
  676.1859,
  606.3408,
  596.5968,
  662.3915,
  656.8872,
  689.4451,
  709.8495,
  697.8015,
  734.9873,
  645.2607,
  721.5644,
  737.9619,
  754.0823,
  750.1808,
  807.1363,
  770.617,
  710.3706,
  758.9639,
  748.2821,
  780.6522,
  783.3917,
  783.359,
  767.8463,
  766.6072,
  797.3362,
  807.1185,
  815.3362,
  794.2155,
  781.0969,
  808.8165,
  758.2287,
  798.1355,
  811.5401,
  781.5107,
  792.4049,
  825.5939,
  804.6078,
  819.8282,
  838.9502,
  809.7674,
  801.2631,
  853.3356,
  821.6568,
  852.9136,
  832.5482,
  857.3293,
  839.1373,


In [10]:
import os
import logging
import shutil
import tempfile
import json
from pathlib import Path

import matlab.engine
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

# --- Configuration ---
BASE_DIR = Path("./")
MODTRAN_MATLAB_DIR = BASE_DIR / "MODTRAN5" / "matlab-modtran-5-aba70d781805"
SCOPE_OUTPUT_FILE = BASE_DIR / "synthetic_dataset" / "results.json"
OUTPUT_DIR = BASE_DIR / "output"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# --- Logging Setup ---
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s [%(levelname)s] %(message)s",
                    handlers=[logging.StreamHandler()])
logger = logging.getLogger(__name__)

# --- MODTRAN Configuration ---
MODTRAN_DIR = Path("/home/omirako/Documents/MSc_Sensors_Imaging/Final_Project/MODTRAN5").resolve()
logger.info(f"Looking for MODTRAN at: {MODTRAN_DIR}")
if not MODTRAN_DIR.exists():
    raise FileNotFoundError(f"MODTRAN directory not found: {MODTRAN_DIR}")

MODTRAN_EXE_DIR = MODTRAN_DIR / "bin"
if not MODTRAN_EXE_DIR.exists():
    raise FileNotFoundError(f"MODTRAN executable directory not found: {MODTRAN_EXE_DIR}")

possible_data_dirs = [MODTRAN_EXE_DIR / "DATA", MODTRAN_DIR / "DATA", MODTRAN_DIR / "data"]
MODTRAN_DATA_DIR = next((d for d in possible_data_dirs if d.exists()), None)
if MODTRAN_DATA_DIR:
    logger.info(f"Found MODTRAN data directory at: {MODTRAN_DATA_DIR}")
else:
    raise FileNotFoundError(f"MODTRAN data directory not found. Searched in: {possible_data_dirs}")

# Check for a band model file (ignored in our case)
possible_bmfiles = [MODTRAN_DATA_DIR / "p1_2013.bin", MODTRAN_DATA_DIR / "p1_2009t.bin",
                    MODTRAN_DATA_DIR / "p1_2009c.bin", MODTRAN_DATA_DIR / "p1_2009.bin"]
BMNAME_FILE = next((f for f in possible_bmfiles if f.exists()), None)
if BMNAME_FILE:
    logger.info(f"Found band model: {BMNAME_FILE.name}, but it will be ignored.")
else:
    logger.info("No band model file found. Proceeding with MODTRAN defaults.")

# --- Start MATLAB Engine ---
logger.info("Starting MATLAB Engine...")
eng = matlab.engine.start_matlab("-nojvm -nodisplay -nosplash")

# --- Configure MATLAB Paths ---
eng.addpath(str(MODTRAN_DIR.resolve()), nargout=0)
eng.addpath(eng.genpath(str(MODTRAN_MATLAB_DIR.resolve())), nargout=0)
eng.addpath(str(MODTRAN_DATA_DIR.resolve()), nargout=0)

import platform
os_type = platform.system()
if os_type == "Windows":
    modtran_exe = MODTRAN_EXE_DIR / "Mod5_win64.exe"
elif os_type == "Linux":
    modtran_exe = MODTRAN_EXE_DIR / "Mod5_linux.exe"
    if not modtran_exe.exists():
        modtran_exe = MODTRAN_EXE_DIR / "Mod5_linux.exe"

# Compute relative paths for flexibility
modtran_matlab_dir = MODTRAN_MATLAB_DIR.resolve()
modtran_exe_dir = MODTRAN_EXE_DIR.resolve()
modtran_exe_path = modtran_exe.resolve()
relative_modtran_path = os.path.relpath(str(modtran_exe_dir), str(modtran_matlab_dir))
relative_modtran_exe = os.path.relpath(str(modtran_exe_path), str(modtran_matlab_dir))

# --- Create temporary MATLAB function to override GUI dialog ---
temp_script = f"""
function [MODTRANExe, MODTRANPath] = SetMODTRANExe()
    MODTRANExe = '{modtran_exe.name}';
    MODTRANPath = '{str(MODTRAN_EXE_DIR.resolve()).replace(os.sep, '/')}/';
end
"""
temp_dir = Path(tempfile.mkdtemp())
script_path = temp_dir / "SetMODTRANExe.m"
with open(script_path, 'w') as f:
    f.write(temp_script)
eng.addpath(str(temp_dir), nargout=0)
eng.eval("Mod5.SetMODTRANExe = @SetMODTRANExe;", nargout=0)
eng.workspace["MODTRANPath"] = relative_modtran_path.replace(os.sep, '/') + "/"
eng.workspace["MODTRANExe"] = relative_modtran_exe.replace(os.sep, '/')

logger.info("Final MATLAB configuration complete.")
try:
    modtran_path = eng.workspace['MODTRANPath']
except Exception:
    modtran_path = "Not Set"
logger.info(f"MODTRANPath in MATLAB: {modtran_path}")
try:
    modtran_exe_ws = eng.workspace['MODTRANExe']
except Exception:
    modtran_exe_ws = "Not Set"
logger.info(f"MODTRANExe in MATLAB: {modtran_exe_ws}")
logger.info(f"Is Mod5.m found? {eng.exist('Mod5.m', 'file')}")
logger.info(f"Is run_MODTRAN5_AC_PAR_4RUNS.m found? {eng.exist('run_MODTRAN5_AC_PAR_4RUNS.m', 'file')}")

eng.addpath(str(MODTRAN_MATLAB_DIR.resolve()), nargout=0)
eng.addpath(eng.genpath(str((MODTRAN_MATLAB_DIR.parent / "modtran5_acd_t14").resolve())), nargout=0)
logger.info("MATLAB paths added successfully.")

# --- Helper Functions ---
def run_modtran_matlab(parms, output_dir, case_name):
    """Runs MODTRAN via MATLAB Engine with proper initialization."""
    try:
        eng.eval("warning('off', 'Mod5:setBMNAME:FileNotFound');", nargout=0)
        eng.workspace['NSTR'] = 1  # Force single-threaded execution
        eng.eval("delete(gcp('nocreate'));", nargout=0)
        matlab_parms = eng.struct()
        for key, value in parms.items():
            if key == 'ATM':
                atm_struct = eng.struct()
                for sub_key, sub_value in value.items():
                    if isinstance(sub_value, (list, np.ndarray)):
                        atm_struct[sub_key] = matlab.double([float(x) for x in sub_value])
                    elif isinstance(sub_value, (int, float)):
                        atm_struct[sub_key] = float(sub_value)
                    else:
                        atm_struct[sub_key] = sub_value
                matlab_parms[key] = atm_struct
            elif key == 'SPECTRAL':
                spectral_struct = eng.struct()
                for sub_key, sub_value in value.items():
                    if sub_key in ('V1', 'V2'):
                        spectral_struct[sub_key] = matlab.double([float(sub_value)])
                    elif isinstance(sub_value, (int, float)):
                        spectral_struct[sub_key] = float(sub_value)
                    else:
                        spectral_struct[sub_key] = sub_value
                matlab_parms[key] = spectral_struct
            else:
                if isinstance(value, (list, np.ndarray)):
                    matlab_parms[key] = matlab.double([float(x) for x in value])
                elif isinstance(value, (int, float)):
                    matlab_parms[key] = float(value)
                else:
                    matlab_parms[key] = value
        eng.cd(str(MODTRAN_MATLAB_DIR.resolve()), nargout=0)
        wvlLUT, T14, _ = eng.run_MODTRAN5_AC_PAR_4RUNS(
            str(output_dir),
            case_name,
            matlab_parms,
            'FLUO',
            'A',
            1,
            0,
            0,
            nargout=3
        )
        return np.array(wvlLUT), np.array(T14)
    except Exception as e:
        logger.error(f"MODTRAN execution failed for {case_name}: {e}")
        return None, None

def interpolate_and_combine(wl_scope, data_scope, wl_modtran):
    """Interpolates SCOPE data to MODTRAN wavelengths."""
    interpolator = interp1d(wl_scope, data_scope, kind='linear', bounds_error=False, fill_value="extrapolate")
    return interpolator(wl_modtran)

def validate_modtran_params(parms):
    required_atm = ['WVL', 'WNDFNM', 'WLINC', 'MODEL', 'H1', 'SZA', 'IHAZE', 'VIS']
    for param in required_atm:
        if param not in parms['ATM']:
            raise ValueError(f"Missing required ATM parameter: {param}")
    required_top_level = ['SPECTRAL', 'GEOM']
    for param in required_top_level:
        if param not in parms:
            raise ValueError(f"Missing required top-level parameter: {param}")
    numerical_params = {
        'ATM': ['AOT', 'H1', 'SZA', 'VZA', 'GNDALT', 'WLINC','VIS'],
        'SPECTRAL': ['V1', 'V2', 'DV', 'FWHM'],
        'GEOM': ['H1ALT', 'OBSZEN'],
        'top_level': ['SFWHM', 'CO2MX', 'RAINRT', 'GNDALT','PARM1','PARM2']
    }
    for category, params in numerical_params.items():
        for param in params:
            if category == 'ATM':
                value = parms['ATM'].get(param)
            elif category == 'SPECTRAL':
                value = parms['SPECTRAL'].get(param)
            elif category == 'GEOM':
                value = parms['GEOM'].get(param)
            else:
                value = parms.get(param)
            if value is None:
                raise ValueError(f"Parameter {param} is required but has no value assigned")
            if isinstance(value, (list, np.ndarray)):
                if not all(isinstance(x, (int, float)) for x in value):
                    raise TypeError(f"Elements of parameter {param} must be numeric")
            elif not isinstance(value, (int, float)):
                raise TypeError(f"Parameter {param} must be numeric, got {type(value)}")

def make_hashable(o):
    """Recursively convert a dict or list into a hashable type (tuple)."""
    if isinstance(o, dict):
        return tuple(sorted((k, make_hashable(v)) for k, v in o.items()))
    elif isinstance(o, list):
        return tuple(make_hashable(x) for x in o)
    else:
        return o

# --- Main Processing ---
if __name__ == "__main__":
    # Load SCOPE Results
    try:
        with open(SCOPE_OUTPUT_FILE, 'r') as f:
            scope_results = json.load(f)
    except Exception as e:
        logger.error(f"Error loading SCOPE results: {e}")
        exit(1)

    # Save common lookup tables (assumed common across all simulations)
    global_lookup = {
        "SCOPE_wlS": scope_results.get("wlS", []),
        "SCOPE_wlF": scope_results.get("wlF", [])
    }
    global_lookup_file = OUTPUT_DIR / "simulation_lookuptable.json"
    with open(global_lookup_file, "w") as f:
        json.dump(global_lookup, f, indent=2)
    logger.info(f"Saved global lookup table to {global_lookup_file}")

    num_simulations = scope_results.get('num_simulations', 0)
    logger.info(f"Number of SCOPE simulations: {num_simulations}")

    # For each simulation, extract SCOPE conditions if they exist.
    # These SCOPE parameters (if provided) are used to override ambient conditions.
    # These SCOPE parameters are stored in run_parameters.
    run_params = scope_results["run_parameters"]

    # Get the overall experiment run parameters from the "input_parameters" field.
    # This defines the full grid of experiments (e.g., if Cab has 3 values, there will be 3 simulations).
    input_parameters = scope_results.get("input_parameters", {})

    # Prepare SCOPE spectral data arrays
    wlS = np.array(scope_results['wlS'])[0]  # assume a single list
    wlF = np.array(scope_results['wlF'])[0]
    reflectance_data = np.array(scope_results['spectral_outputs']['reflectance'])
    fluorescence_data = np.array(scope_results['spectral_outputs']['fluorescence'])
    
    # Initialize cache to store unique MODTRAN simulation results.
    modtran_cache = {}
    modtran_results = []  # To store details if needed
    global_lookup_saved = False

    # Loop over each SCOPE simulation and each ambient condition variant.
    for i in range(num_simulations):
        # Extract SCOPE run parameters for simulation i.
        current_scope = run_params[i]

        # Base shared ambient conditions (these remain fixed between MODTRAN and SCOPE)
        base_shared = {
            "SZA": current_scope.get("Angles.tts", 30.0),    # Solar zenith angle (degrees),
            "TPTEMP": current_scope.get("Meteo.Ta", 20.0),   # Air temperature (°C),
            "CO2": current_scope.get("Meteo.Ca", 410.0),     # CO₂ mixing ratio (ppm),
            "psi": current_scope.get("Angles.psi", 0.0),       # Solar/sensor azimuth angle (degrees),
            "tto": current_scope.get("Angles.tto", 0.0),       # Observation zenith angle (degrees),
        }

        # Create ambient conditions (base case and various perturbed cases)
        ambient_conditions = [
            base_shared.copy(),                                      # Base case: shared parameters only
            dict(base_shared, O3=0.3, NO2=0.1, CO=0.1),              # Perturbed case with O₃, NO₂, CO specified
            dict(base_shared, O3=0.35),                              # Case with modified O₃ level
            dict(base_shared, NO2=0.15),                             # Case with modified NO₂ level
            dict(base_shared, CO=0.2),                               # Case with modified CO level
            dict(base_shared, O3=0.25, NO2=0.05),                    # Case with lower O₃ and NO₂
            dict(base_shared, O3=0.3, CO=0.15),                      # Case with O₃ and CO modifications
            dict(base_shared, NO2=0.1, CO=0.05),                     # Case with NO₂ and CO modifications
            dict(base_shared, O3=0.2, NO2=0.2),                      # Case with decreased O₃ and increased NO₂
            dict(base_shared, O3=0.3, NO2=0.1, CO=0.2),              # Case with all three gases modified
        ]

        # Process each ambient condition variant for the current simulation
        for j, ambient in enumerate(ambient_conditions):
            # Build the effective MODTRAN parameters dictionary (PARMS) using ambient values.
            # Values are taken from the ambient condition if provided,
            # otherwise from the simulation defaults extracted above.
            PARMS = {
                # Top-level simulation controls:
                'CASE':      ambient.get("CASE", 0),                # Simulation case identifier (default: 0)
                'MODTRN':    ambient.get("MODTRN", 'M'),            # MODTRAN mode ('M' for standard mode)
                'SPEED':     ambient.get("SPEED", 'S'),             # Simulation speed option
                'LYMOLC':    ambient.get("LYMOLC", 0),              # Molecule-specific option (default: 0)
                'MODEL':     ambient.get("MODEL", 2),               # Atmospheric model selection (default: 2)
                'ITYPE':     ambient.get("ITYPE", 3),               # Instrument/observation type (default: 3)
                'IEMSCT':    ambient.get("IEMSCT", 3),              # Scattering option flag
                'IMULT':     ambient.get("IMULT", 0),               # Multiplicative flag (default: 0)
                'M1':        ambient.get("M1", 0),                  # Additional model parameter (default: 0)
                'M2':        ambient.get("M2", 0),                  # Additional model parameter (default: 0)
                'M3':        ambient.get("M3", 0),                  # Additional model parameter (default: 0)
                'M4':        ambient.get("M4", 0),                  # Additional model parameter (default: 0)
                'M5':        ambient.get("M5", 0),                  # Additional model parameter (default: 0)
                'M6':        ambient.get("M6", 0),                  # Additional model parameter (default: 0)
                'MDEF':      ambient.get("MDEF", 0),                # Model definition flag (default: 0)
                'I_RD2C':    ambient.get("I_RD2C", 0),              # Radiative transfer option (default: 0)
                'NOPRNT':    ambient.get("NOPRNT", 0),              # Print output flag (default: 0)
                'TPTEMP':    ambient.get("TPTEMP", base_shared["TPTEMP"]),  # Temperature for Planck function (°C; shared "Ta")
                'SURREF':    ambient.get("SURREF", 'LAMB_1'),       # Surface reflection model (e.g., Lambertian)
                'DIS':       ambient.get("DIS", '1'),               # Display/output option (default: '1')
                'DISAZM':    ambient.get("DISAZM", '0'),            # Azimuth display flag (default: '0')
                'DISALB':    ambient.get("DISALB", ' '),            # Albedo display placeholder
                'NSTR':      ambient.get("NSTR", 8),                # Number of radiative transfer streams
                'SFWHM':     ambient.get("SFWHM", 1.0),             # Instrument spectral response FWHM
                'CO2MX':     ambient.get("CO2", base_shared["CO2"]),  # CO₂ mixing ratio (ppm; shared "CO2")
                'LSUNFL':    ambient.get("LSUNFL", 1),              # Sun flag: include direct sun (default: 1)
                'LBMNAM':    ambient.get("LBMNAM", 0),              # Disable band model lookup (default: 0)
                'BMNAME':    ambient.get("BMNAME", ' '),            # Band model name (default: blank)
                'IDAY':      ambient.get("IDAY", 191),              # Day of year for simulation
                'GMTIME':    ambient.get("GMTIME", 12.0),           # Greenwich Mean Time (hours)
                'NSSALB':    ambient.get("NSSALB", 0),              # Spherical albedo placeholder
                'IPARM':     ambient.get("IPARM", 11),              # Parameter flag (default: 11)
                'APLUS':     ambient.get("APLUS", ' '),             # Additional parameter placeholder
                'CNOVAM':    ambient.get("CNOVAM", ' '),            # Additional parameter placeholder
                'ARUSS':     ambient.get("ARUSS", ' '),             # Additional parameter placeholder
                'SOLCON':    ambient.get("SOLCON", ' '),            # Solar constant correction placeholder
                'LFLTNM':    ambient.get("LFLTNM", ' '),            # Output filename placeholder
                'WSS':       ambient.get("WSS", 0.0),               # Wind speed (default: 0)
                'WDS':       ambient.get("WDS", 0.0),               # Wind direction (default: 0)
                'VIS':       ambient.get("VIS", 23.0),              # Visibility in km (affects scattering)
                'RAINRT':    ambient.get("RAINRT", 0.0),            # Rain rate (mm/hr; default: 0)
                'GNDALT':    ambient.get("GNDALT", 1),              # Ground altitude (default: 1km)
                'T_BEST':    ambient.get("T_BEST", ' '),            # Best-fit temperature placeholder

                # Atmospheric (ATM) parameters:
                'ATM': {
                    'H1':     ambient.get("H1", 1),             # Observer altitude (default: ground altitude 1km)
                    'GNDALT': ambient.get("GNDALT", 1),         # Ground altitude (default: 1km)
                    'SZA':    ambient.get("SZA", base_shared["SZA"]),  # Solar Zenith Angle (shared SZA)
                    'SAA':    ambient.get("SAA", 180.0 - base_shared["psi"]),  # Solar Azimuth Angle (derived from shared psi)
                    'RAA':    ambient.get("RAA", 0.0),              # Relative Azimuth Angle (default: 0)
                    'VZA':    ambient.get("VZA", base_shared["tto"]),  # Observer Zenith Angle (derived from shared tto)
                    'MODEL':  ambient.get("ATM_MODEL", 2),          # Atmospheric model (default: 2)
                    'IDAY':   ambient.get("ATM_IDAY", 191),          # Day of year for atmospheric simulation
                    'G':      ambient.get("G", 0.7),                # Asymmetry factor for scattering (default: 0.7)
                    'AOT':    ambient.get("AOT", 0.1),              # Aerosol Optical Thickness (default: 0.1)
                    'CDASTM': ambient.get("CDASTM", ' '),           # Aerosol/storm parameters placeholder
                    'ASTMX':  ambient.get("ASTMX", 0.0),            # Maximum aerosol optical thickness (default: 0)
                    'CMULT':  ambient.get("CMULT", 1.0),            # Multiplicative factor for aerosol correction (default: 1.0)
                    'IHAZE':  ambient.get("IHAZE", 1),              # Aerosol model flag (default: 1)
                    'H2OSTR': ambient.get("H2O", 2.0),             # Water vapor content (default: 2.0)
                    'O3STR':  str(ambient.get("O3", 'DEFAULT')),    # Ozone parameter (ambient or default)
                    'C_PROF': ambient.get("C_PROF", ' '),          # Atmospheric profile placeholder
                    'H2OAER': ambient.get("H2OAER", ' '),          # Aerosol water content placeholder
                    'CDTDIR': ambient.get("CDTDIR", ' '),          # Direct irradiance component placeholder
                    'ASTMO':  ambient.get("ASTMO", ' '),            # Additional aerosol options placeholder
                    'AERRH':  ambient.get("AERRH", ' '),            # Aerosol error term placeholder
                    'IPH':    ambient.get("IPH", 2),               # Photon counting flag (default: 2)
                    'ISEASN': ambient.get("ISEASN", 0),             # Additional aerosol parameter (default: 0)
                    'IVULCN': ambient.get("IVULCN", 0),             # Additional aerosol parameter (default: 0)
                    'ICSTL':  ambient.get("ICSTL", 1),              # Cloud/scattering type flag (default: 1)
                    'ICLD':   ambient.get("ICLD", 0),               # Cloudiness flag (default: 0)
                    'IVSA':   ambient.get("IVSA", 0),               # Additional scattering parameter (default: 0)
                    'WVL':    ambient.get("WVL", [650.0, 850.0]),    # Wavelength range (nm) for simulation
                    'WNDFNM': ambient.get("WNDFNM", [650.0, 850.0]), # Wavelength range for lookup table (same as WVL)
                    'WLINC':  ambient.get("WLINC", 1.0),             # Wavelength increment (nm; smaller means higher resolution)
                    'VIS':    ambient.get("ATM_VIS", 23.0),          # Atmospheric visibility (km)
                },

                # Geometric (GEOM) parameters:
                'GEOM': {
                    'ITYPE': ambient.get("GEOM_ITYPE", 3),         # Geometric/observation type (default: 3)
                    'H1ALT': ambient.get("H1ALT", 500),             # Sensor altitude (default: 500)
                    'OBSZEN': ambient.get("OBSZEN", 180.0 - base_shared["tto"]),  # Observer zenith angle (derived from shared tto)
                    'TRUEAZ': ambient.get("TRUEAZ", base_shared["psi"] + 180.0),  # True azimuth angle (derived from shared psi)
                },

                # Additional top-level parameters:
                'PARM1': ambient.get("PARM1", ambient.get("SZA", base_shared["SZA"])),         # Parameter 1 (default: SZA)
                'PARM2': ambient.get("PARM2", 0.0),         # Parameter 2 (unused; default: 0)

                # Spectral configuration parameters:
                'SPECTRAL': {
                    'V1': ambient.get("V1", 1e7 / 850),   # Starting wavenumber (calculated from 850 nm)
                    'V2': ambient.get("V2", 1e7 / 650),   # Ending wavenumber (calculated from 650 nm)
                    'DV': ambient.get("DV", 20.0),         # Spectral resolution (cm⁻¹; lower DV gives higher resolution)
                    'FWHM': ambient.get("FWHM", 0.27),     # Instrument spectral response FWHM, similar to the paper
                    'CM_1': ambient.get("CM_1", True),     # Flag indicating if the spectral range is in cm⁻¹ (True = yes)
                },
            }

            # Validate parameters
            try:
                validate_modtran_params(PARMS)
            except Exception as e:
                logger.error(f"Validation failed for simulation sim_{i}_amb_{j}: {e}")
                continue

            # Create a unique hashable key from the PARMS dictionary.
            modtran_key = make_hashable(PARMS)
            case_name = f"sim_{i}_amb_{j}"

            if modtran_key in modtran_cache:
                logger.info(f"Reusing cached MODTRAN simulation for {case_name}")
                wvlLUT, T14 = modtran_cache[modtran_key]
            else:
                logger.info(f"Running MODTRAN for {case_name}")
                wvlLUT, T14 = run_modtran_matlab(PARMS, str(OUTPUT_DIR), case_name)
                if wvlLUT is None:
                    logger.error(f"Skipping {case_name} due to MODTRAN error.")
                    continue
                modtran_cache[modtran_key] = (wvlLUT, T14)
                modtran_results.append({
                    "case_name": case_name,
                    "PARMS": PARMS,
                    "wavelength_modtran": wvlLUT.tolist()
                })

            # Ensure wvlLUT is one-dimensional
            wvlLUT = np.array(wvlLUT).flatten()

            # Interpolate SCOPE outputs (reflectance and fluorescence) to MODTRAN wavelengths.
            idx_ref = i % reflectance_data.shape[0]
            R = interpolate_and_combine(wlS, reflectance_data[idx_ref, :], wvlLUT).flatten()
            F = interpolate_and_combine(wlF, fluorescence_data[idx_ref, :], wvlLUT).flatten()

            # Calculate LBOA and LWLR to obtain the apparent reflectance ρ_app per Equation 5.
            t_vals = {f't{j}': T14[:, j-1] for j in range(1, 13)}
            t1 = t_vals['t1']
            t2 = t_vals['t2']
            t3 = t_vals['t3']
            t4 = t_vals['t4'] + t_vals['t5']  # sum of t4 and t5
            t6 = t_vals['t6']
            t7 = t_vals['t7']
            t8 = t_vals['t8']
            t9 = t_vals['t9']
            t10 = t_vals['t10']
            t11 = t_vals['t11']

            LBOA = t1 * t4 * R + t1 * ((t9 * R + t10 * R + t11 * R) / (1 - R * t3)) + t1 * (t6 * F + t7 * F)
            LWLR = t1 * t4 + t1 * ((t9 * R + t10 * R + t11 * R) / (1 - R * t3))
            rho_app = LBOA / LWLR

            #LTOA = t1 * t2 + (t1 * t8 * rho_app + t9 * rho_app + t10 * rho_app + t11 * rho_app + t6 * F + t7 * F) / (1 - t3 * rho_app) 
            LTOA = t1 * t2 + (t1 * t8 * R + t9 * R + t10 * R + t11 * R + t6 * F + t7 * F) / (1 - t3 * R)
            #LTOA = t1 * t2 + (t1 * t8 * rho_app + t9 * rho_app + t10 * rho_app + t11 * rho_app + t6 * F + t7 * F) / (1 - t3 * R) 
            # divide the denominator in reflected radiance and fluorescence variance, the reflectance one has to be similar to
            print("wvlLUT shape:", wvlLUT.shape)
            print("LTOA shape:", LTOA.shape)
            
            if not global_lookup_saved:
                global_lookup = {
                    "SCOPE_wlS": wlS.tolist(),
                    "SCOPE_wlF": wlF.tolist(),
                    "modtran_wavelength": wvlLUT.tolist()
                }
                with open(global_lookup_file, "w") as f:
                    json.dump(global_lookup, f, indent=2)
                logger.info(f"Saved global lookup table to {global_lookup_file}")
                global_lookup_saved = True

            # Build a dictionary with all desired simulation-specific results
            sim_results = {
                "case_name": case_name,
                "SCOPE_settings": current_scope,         # settings for this simulation from results.json
                "MODTRAN_settings": PARMS,               # the MODTRAN input parameters used
                "wavelength_modtran": wvlLUT.tolist(),   # MODTRAN lookup table (if not common, otherwise omit)
                "LTOA": LTOA.tolist(),                   # computed top-of-atmosphere radiance
                #"T14": T14.tolist(),                     # full T14 matrix from MODTRAN
                "t_vals": {                            # transfer function values, as lists
                    key: t_vals[key].tolist() for key in t_vals
                },
                "rho_app": rho_app.tolist(),             # computed apparent reflectance
                "R": R.tolist(),                         # interpolated reflectance from SCOPE
                "F": F.tolist(),                         # interpolated fluorescence from SCOPE
                "Esun": scope_results['spectral_outputs']['Esun'][idx_ref],  # solar irradiance
                # Instead of storing SCOPE wavelength grids in each file,
                # we add a reference to the global lookup table file.
                "lookup_table_reference": str(global_lookup_file)
                # You can also store the specific SCOPE reflectance and fluorescence row if needed:
                # "SCOPE_reflectance": reflectance_data[idx_ref, :].tolist(),
                # "SCOPE_fluorescence": fluorescence_data[idx_ref, :].tolist()
            }

            # Save the simulation results to a JSON file for this case.
            output_json_file = OUTPUT_DIR / f"simulation_{case_name}.json"
            with open(output_json_file, "w") as f:
                json.dump(sim_results, f, indent=2)
            logger.info(f"Saved simulation results for {case_name} to {output_json_file}")

    # Quit MATLAB engine and clean up temporary directory
    eng.quit()
    shutil.rmtree(temp_dir)
    logger.info("MATLAB Engine quit and temporary files removed.")

    # Update SCOPE results JSON with MODTRAN simulation details.
    scope_results.setdefault("modtran_simulations", [])
    scope_results["modtran_simulations"].extend(modtran_results)
    with open(SCOPE_OUTPUT_FILE, "w") as f:
        json.dump(scope_results, f, indent=2)
    logger.info(f"Updated results saved to {SCOPE_OUTPUT_FILE}")

    logger.info("Processing complete.")


2025-03-21 16:09:03,849 [INFO] Looking for MODTRAN at: /home/omirako/Documents/MSc_Sensors_Imaging/Final_Project/MODTRAN5
2025-03-21 16:09:03,850 [INFO] Found MODTRAN data directory at: /home/omirako/Documents/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/DATA
2025-03-21 16:09:03,851 [INFO] Found band model: p1_2009t.bin, but it will be ignored.
2025-03-21 16:09:03,852 [INFO] Starting MATLAB Engine...
2025-03-21 16:09:05,959 [INFO] Final MATLAB configuration complete.
2025-03-21 16:09:05,967 [INFO] MODTRANPath in MATLAB: ../bin/
2025-03-21 16:09:05,971 [INFO] MODTRANExe in MATLAB: ../bin/Mod5_linux.exe
2025-03-21 16:09:05,978 [INFO] Is Mod5.m found? 2.0
2025-03-21 16:09:05,986 [INFO] Is run_MODTRAN5_AC_PAR_4RUNS.m found? 2.0
2025-03-21 16:09:06,031 [INFO] MATLAB paths added successfully.
2025-03-21 16:09:06,102 [INFO] Saved global lookup table to output/simulation_lookuptable.json
2025-03-21 16:09:06,102 [INFO] Number of SCOPE simulations: 135
2025-03-21 16:09:06,110 [INFO] Running MO

--- Starting run_MODTRAN5_AC_PAR_4RUNS at 21-Mar-2025 16:09:06 ---
MODTRAN5 - MIT 4 RUNS method (parallel version)
Initializing simulation parameters...
Cleared previous MODTRAN classes.
Atmospheric and geometric parameters set.
Loading tp5 base file for SENSOR: FLUO
TP5 base file loaded.
Base file parameters configured.
**************************************
MODTRAN5 is running in 1cm-1 configuration
**************************************
Output directory created at output/T14-Modtran
Initialized MODTRAN spectral output array.
Writing TP5 base file to: output/T14-Modtran/sim_0_amb_0.tp5
TP5 base file written.
Initializing MODTRAN5 cases...
MODTRAN5 cases initialized.
Loading MODTRANExe settings...
Using MODTRANPath: /home/omirako/Documents/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/
Using MODTRANExe: /home/omirako/Documents/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_linux.exe
Starting parallel pool (parpool) using the 'local' profile ...
Connected to parallel pool with 4 

2025-03-21 16:09:23,416 [INFO] Saved global lookup table to output/simulation_lookuptable.json
2025-03-21 16:09:23,466 [INFO] Saved simulation results for sim_0_amb_0 to output/simulation_sim_0_amb_0.json
2025-03-21 16:09:23,466 [INFO] Running MODTRAN for sim_0_amb_1


Parallel pool using the 'Processes' profile is shutting down.
--- Starting run_MODTRAN5_AC_PAR_4RUNS at 21-Mar-2025 16:09:25 ---
MODTRAN5 - MIT 4 RUNS method (parallel version)
Initializing simulation parameters...
Cleared previous MODTRAN classes.
Atmospheric and geometric parameters set.
Loading tp5 base file for SENSOR: FLUO
TP5 base file loaded.
Base file parameters configured.
**************************************
MODTRAN5 is running in 1cm-1 configuration
**************************************
Output directory created at output/T14-Modtran
Initialized MODTRAN spectral output array.
Writing TP5 base file to: output/T14-Modtran/sim_0_amb_1.tp5
TP5 base file written.
Initializing MODTRAN5 cases...
MODTRAN5 cases initialized.
Loading MODTRANExe settings...
Using MODTRANPath: /home/omirako/Documents/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/
Using MODTRANExe: /home/omirako/Documents/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_linux.exe
Starting parallel pool (parpool) us

2025-03-21 16:09:39,990 [INFO] Saved simulation results for sim_0_amb_1 to output/simulation_sim_0_amb_1.json
2025-03-21 16:09:39,990 [INFO] Running MODTRAN for sim_0_amb_2


...

2025-03-21 16:13:57,326 [INFO] Saved simulation results for sim_134_amb_0 to output/simulation_sim_134_amb_0.json
2025-03-21 16:13:57,326 [INFO] Reusing cached MODTRAN simulation for sim_134_amb_1
2025-03-21 16:13:57,363 [INFO] Saved simulation results for sim_134_amb_1 to output/simulation_sim_134_amb_1.json
2025-03-21 16:13:57,363 [INFO] Reusing cached MODTRAN simulation for sim_134_amb_2
2025-03-21 16:13:57,400 [INFO] Saved simulation results for sim_134_amb_2 to output/simulation_sim_134_amb_2.json
2025-03-21 16:13:57,400 [INFO] Reusing cached MODTRAN simulation for sim_134_amb_3
2025-03-21 16:13:57,436 [INFO] Saved simulation results for sim_134_amb_3 to output/simulation_sim_134_amb_3.json
2025-03-21 16:13:57,437 [INFO] Reusing cached MODTRAN simulation for sim_134_amb_4
2025-03-21 16:13:57,473 [INFO] Saved simulation results for sim_134_amb_4 to output/simulation_sim_134_amb_4.json
2025-03-21 16:13:57,474 [INFO] Reusing cached MODTRAN simulation for sim_134_amb_5
2025-03-21 16:1

wvlLUT shape: (3620,)
LTOA shape: (3620,)
wvlLUT shape: (3620,)
LTOA shape: (3620,)
wvlLUT shape: (3620,)
LTOA shape: (3620,)
wvlLUT shape: (3620,)
LTOA shape: (3620,)
wvlLUT shape: (3620,)
LTOA shape: (3620,)
wvlLUT shape: (3620,)
LTOA shape: (3620,)


2025-03-21 16:13:57,547 [INFO] Saved simulation results for sim_134_amb_6 to output/simulation_sim_134_amb_6.json
2025-03-21 16:13:57,547 [INFO] Reusing cached MODTRAN simulation for sim_134_amb_7
2025-03-21 16:13:57,584 [INFO] Saved simulation results for sim_134_amb_7 to output/simulation_sim_134_amb_7.json
2025-03-21 16:13:57,584 [INFO] Reusing cached MODTRAN simulation for sim_134_amb_8
2025-03-21 16:13:57,621 [INFO] Saved simulation results for sim_134_amb_8 to output/simulation_sim_134_amb_8.json
2025-03-21 16:13:57,621 [INFO] Reusing cached MODTRAN simulation for sim_134_amb_9
2025-03-21 16:13:57,657 [INFO] Saved simulation results for sim_134_amb_9 to output/simulation_sim_134_amb_9.json
2025-03-21 16:13:57,659 [INFO] MATLAB Engine quit and temporary files removed.


wvlLUT shape: (3620,)
LTOA shape: (3620,)
wvlLUT shape: (3620,)
LTOA shape: (3620,)
wvlLUT shape: (3620,)
LTOA shape: (3620,)


2025-03-21 16:13:58,041 [INFO] Updated results saved to synthetic_dataset/results.json
2025-03-21 16:13:58,042 [INFO] Processing complete.
